# 모듈 Import

In [1]:
import cv2 as cv
import sys
import os
from matplotlib import pyplot as plt

import numpy as np
from tensorflow.keras import models, layers, optimizers, utils, Model, regularizers
from keras.utils import np_utils #to_categorical
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score,f1_score,recall_score

from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import *

In [21]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"

# 얼굴 Dataset

In [2]:
path='C:/Users/clari/Desktop/multimodal2/model2/face'

face_images = []
labels = []
def crop_images(path, images):
    for img in sorted(os.listdir(path)):
        img_array = cv.imread(os.path.join(path, img))/255
        face_images.append(img_array)
        label = np_utils.to_categorical(int(img[:3]), 64)
        labels.append(label)

crop_images(path, face_images)
face_images = np.array(face_images)
labels = np.array(labels)

In [3]:
from keras.preprocessing.image import ImageDataGenerator

trainDataGen = ImageDataGenerator(
                                 width_shift_range=0.05,
                                 height_shift_range=0.05,
                                 brightness_range=[0.7, 1.3],
                                 horizontal_flip=True,
                                 )

In [4]:
path2='C:/Users/clari/Desktop/multimodal2/model2/aug/face_aug_small2'

In [15]:
i = 0
j = 1
num=4 #레이블 당 개수
im=[]

for img in face_images :
    im.append(img)
    im=np.array(im)
    for batch in trainDataGen.flow(im, batch_size=1, save_to_dir=path2,
                                   save_prefix=str(format(int(i/num),'03')), save_format='png'):
        j += 1
        if j > 7:
            break
    j = 1
    i+=1
    im=[]

In [5]:
def append_images(path, images, labels):
    a=1
    for img in os.listdir(path):
        img_array = cv.imread(os.path.join(path, img))
        images = np.append(images, [img_array], axis = 0)
        label = np_utils.to_categorical(int(img[:3]), 64)
        labels = np.append(labels, [label], axis=0)
    return images, labels
        
face_images, labels = append_images(path2, face_images, labels)

# 홍채 Dataset

In [6]:
path='C:/Users/clari/Desktop/multimodal2/model2/iris'

#786 576
sc=0.1

iris_images = []

def img_trim (img):     
    x = 150; y = 90;
    w = 476; h = 430;
    img_trim = img[y:y+h, x:x+w]
    return img_trim

def crop_images(path, images):
    for img in sorted(os.listdir(path)):
        img_array = cv.imread(os.path.join(path, img))/255.0
        img_array = cv.resize(img_trim(img_array), (0,0), fx=sc, fy=sc)
        iris_images.append(img_array)
    

crop_images(path, iris_images)
iris_images = np.array(iris_images)

In [7]:
trainDataGen2 = ImageDataGenerator(
                                 width_shift_range=0.05,
                                 height_shift_range=0.05,
                                 brightness_range=[0.6, 1.3]
                                # horizontal_flip=True,
                                 )

In [8]:
path3='C:/Users/clari/Desktop/multimodal2/model2/aug/iris_aug_small2'

In [23]:
i = 0
j = 1
im=[]
num=4

for img in iris_images :
    im.append(img)
    im=np.array(im)
    for batch in trainDataGen2.flow(im, batch_size=1, save_to_dir=path3, save_prefix=format(int(i/num),'03'), save_format='png'):
        j += 1
        if j > 7:
            break
    j = 1
    i+=1
    im=[]

  

In [9]:
def append_images(path, images):
    a=1
    for img in os.listdir(path):
        img_array = cv.imread(os.path.join(path, img))
        images = np.append(images, [img_array], axis = 0)
    return images

iris_images = append_images(path3, iris_images)

In [10]:
print(face_images.shape)
print(iris_images.shape)

(2048, 56, 46, 3)
(2048, 43, 48, 3)


In [11]:
#face 112, 92, 3
#iris 129, 143, 3
image = []

for img in face_images : 
    img = cv.resize(img, (43, 48))
    img = img.transpose(1, 0, 2)
    image.append(img) 
    
image = np.array(image)
print(image[0].shape)
print(iris_images[0].shape)

images = []
for i in range(len(image)) :
    images.append(np.concatenate((image[i],iris_images[i]),axis=0))

images = np.array(images)

(43, 48, 3)
(43, 48, 3)


In [12]:
print('images :',images.shape)
print('labels :',labels.shape)

images : (2048, 86, 48, 3)
labels : (2048, 64)


# 모델 정의

In [13]:
input=layers.Input(shape=(86, 48, 3))

x = input
x = layers.Conv2D(filters = 32, kernel_size = (3,3), padding='Same', activation='relu')(x)
x = layers.MaxPooling2D(pool_size = (2,2))(x)
x = layers.Dropout(0.1)(x)

x = layers.Conv2D(filters = 32, kernel_size = (3,3), padding='Same', activation='relu')(x)
x = layers.AveragePooling2D(pool_size = (2,2))(x)
x = layers.Dropout(0.1)(x)

x = layers.Conv2D(filters = 64, kernel_size = (3,3), padding='Same', activation='relu')(x)
x = layers.AveragePooling2D(pool_size = (2,2))(x)
x = layers.Dropout(0.1)(x)

x = layers.Conv2D(filters = 128, kernel_size = (3,3), padding='Same', activation='relu')(x)
x = layers.MaxPooling2D(pool_size = (2,2))(x)

x = layers.Flatten()(x)
x = layers.Dense(50, kernel_initializer='he_normal')(x)
x = layers.BatchNormalization()(x)
x = layers.Dense(64, activation='softmax')(x)
model = Model(inputs= input, outputs=x)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


# 학습

In [14]:
from keras.callbacks import EarlyStopping

earlystopping = EarlyStopping(monitor='loss',     # 모니터 기준 설정 (loss) 
                              patience = 3,         # 5회 Epoch동안 개선되지 않는다면 종료
                             )

#Cross validation
num = 4
cross = KFold(n_splits = num, shuffle = True, random_state = 2021)
accs = [];  precision = [];  recall = [];  F1 = []
history = []
i = 0

for train_index,test_index in cross.split(images, labels):
    #훈련용
    x_train = images[train_index] #(900, 56, 46, 3)
    y_train = labels[train_index] #(900, 350)
    #검증용
    x_test = images[test_index] #(150, 56, 46, 3)
    y_test = labels[test_index] #(150, 350)
    
    i += 1    
    print('Fold' + str(i))
    
    #훈련용에 대해 model 학습시키고()
    history.append(model.fit(x_train, y_train, epochs = 60, batch_size = 32, callbacks = earlystopping))
    print('-------------------------------------------------------------------------------')
        
    y_test = np.argmax(y_test,axis = 1)
    pred = np.argmax(model.predict(x_test), axis = 1)
    
    #test로 model 검증(1)
    accs.append(accuracy_score(y_test, pred))
    precision.append(precision_score(y_test,pred, average = 'macro'))
    recall.append(recall_score(y_test, pred, average = 'macro'))
    F1.append(f1_score(y_test, pred, average = 'macro'))

Fold1
Epoch 1/60
48/48 [==============================] - 11s 170ms/step - loss: 4.2296 - accuracy: 0.0588
Epoch 2/60
48/48 [==============================] - 8s 172ms/step - loss: 2.9481 - accuracy: 0.3441
Epoch 3/60
48/48 [==============================] - 8s 172ms/step - loss: 1.9554 - accuracy: 0.6687
Epoch 4/60
48/48 [==============================] - 8s 172ms/step - loss: 1.3025 - accuracy: 0.8273
Epoch 5/60
48/48 [==============================] - 8s 172ms/step - loss: 0.9723 - accuracy: 0.8569
Epoch 6/60
48/48 [==============================] - 8s 175ms/step - loss: 0.8000 - accuracy: 0.8741
Epoch 7/60
48/48 [==============================] - 8s 175ms/step - loss: 0.6618 - accuracy: 0.8780
Epoch 8/60
48/48 [==============================] - 8s 171ms/step - loss: 0.5148 - accuracy: 0.8964
Epoch 9/60
48/48 [==============================] - 8s 171ms/step - loss: 0.3943 - accuracy: 0.9408
Epoch 10/60
48/48 [==============================] - 8s 172ms/step - loss: 0.2593 - accuracy:

C:\Users\clari\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Fold3
Epoch 1/60
48/48 [==============================] - 8s 168ms/step - loss: 0.0973 - accuracy: 0.9857
Epoch 2/60
48/48 [==============================] - 8s 172ms/step - loss: 0.0304 - accuracy: 0.9954
Epoch 3/60
48/48 [==============================] - 8s 171ms/step - loss: 0.0154 - accuracy: 0.9980
Epoch 4/60
48/48 [==============================] - 8s 171ms/step - loss: 0.0059 - accuracy: 1.0000
Epoch 5/60
48/48 [==============================] - 8s 177ms/step - loss: 0.0036 - accuracy: 1.0000
Epoch 6/60
48/48 [==============================] - 8s 174ms/step - loss: 0.0029 - accuracy: 1.0000
Epoch 7/60
48/48 [==============================] - 8s 170ms/step - loss: 0.0026 - accuracy: 1.0000
Epoch 8/60
48/48 [==============================] - 8s 168ms/step - loss: 0.0024 - accuracy: 1.0000
Epoch 9/60
48/48 [==============================] - 8s 168ms/step - loss: 0.0023 - accuracy: 1.0000
Epoch 10/60
48/48 [==============================] - 8s 165ms/step - loss: 0.0020 - accuracy: 

In [67]:
model.summary()

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 86, 48, 3)]       0         
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 86, 48, 32)        896       
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 43, 24, 32)        0         
_________________________________________________________________
dropout_23 (Dropout)         (None, 43, 24, 32)        0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 43, 24, 32)        9248      
_________________________________________________________________
average_pooling2d_14 (Averag (None, 21, 12, 32)        0         
_________________________________________________________________
dropout_24 (Dropout)         (None, 21, 12, 32)        0   

In [15]:
for i in range(1,num+1):
    print("Fold%d 훈련 결과 : "%i)
    print("     accs : %.6f"%accs[i-1])
    print("     precision : %.6f"%precision[i-1])
    print("     recall : %.6f"%recall[i-1])
    print("     F1 : %.6f"%F1[i-1])

print("\n평균 훈련 결과")
print("     Accuracy : %.6f"%np.mean(accs))
print("     Precision : %.6f"%np.mean(precision))
print("     Recall Score : %.6f"%np.mean(recall))
print("     F1-Score : %.6f"%np.mean(F1))

Fold1 훈련 결과 : 
     accs : 0.994141
     precision : 0.995573
     recall : 0.996801
     F1 : 0.995890
Fold2 훈련 결과 : 
     accs : 0.890625
     precision : 0.916590
     recall : 0.901296
     F1 : 0.880802
Fold3 훈련 결과 : 
     accs : 1.000000
     precision : 1.000000
     recall : 1.000000
     F1 : 1.000000
Fold4 훈련 결과 : 
     accs : 0.996094
     precision : 0.995455
     recall : 0.996205
     F1 : 0.995496

평균 훈련 결과
     Accuracy : 0.970215
     Precision : 0.976904
     Recall Score : 0.973576
     F1-Score : 0.968047


# 훈련 과정 시각화

In [1]:
# summarize history for accuracy
plt.plot(history[0].history['accuracy'])
plt.plot(history[1].history['accuracy'])
plt.plot(history[2].history['accuracy'])
plt.plot(history[3].history['accuracy'])
plt.plot(history[4].history['accuracy'])
plt.title('model train accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['fold1', 'fold2','fold3','fold4','fold5'], loc=0)
plt.show()

# summarize history for accuracy
plt.plot(history[0].history['loss'])
plt.plot(history[1].history['loss'])
plt.plot(history[2].history['loss'])
plt.plot(history[3].history['loss'])
plt.plot(history[4].history['loss'])
plt.title('model train accuracy')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['fold1', 'fold2','fold3','fold4','fold5'], loc='upper left')
plt.show()


NameError: name 'plt' is not defined

# 테스트 데이터셋 예측

In [17]:
path='C:/Users/clari/Desktop/multimodal2/model2/face_test'

test_num = []
face_test_images = []
def crop_test_images(path, images):
    for img in os.listdir(path):
        img_array = cv.imread(os.path.join(path, img))/255.0
        face_test_images.append(img_array)
        test_num.append(int(img[:2]))

crop_test_images(path, face_test_images)

In [18]:
path2='C:/Users/clari/Desktop/multimodal2/model2/iris_test'

iris_test_images = []
def crop_test_images2(path, test_images):
    for img in os.listdir(path):
      img_array = cv.imread(os.path.join(path, img))/255.0
      img_array = cv.resize(img_trim(img_array), (0,0), fx=sc, fy=sc) 
      iris_test_images.append(img_array)

crop_test_images2(path2, iris_test_images)

In [19]:
#face 112, 92, 3
#iris 129, 143, 3
test_image = []

for img in face_test_images : 
    img = cv.resize(img, (43, 48))
    img = img.transpose(1, 0, 2)
    test_image.append(img) 
    
test_image = np.array(test_image)

test_images = []
for i in range(len(test_image)) :
    test_images.append(np.concatenate((test_image[i],iris_test_images[i]),axis=0))

test_images = np.array(test_images)

In [20]:
print(test_images.shape)
print(images.shape)

(64, 86, 48, 3)
(2048, 86, 48, 3)


In [21]:
import pandas as pd

pred = np.argmax(model.predict(test_images), axis=1)

df = pd.DataFrame({'Image': test_num, 'Answer': pred}).sort_values('Image')
df.to_csv('C:/Users/clari/Desktop/multimodal2/model1/멀티모달_모델1개.csv', index = False)